이 노트북은 [케라스 창시자에게 배우는 딥러닝 2판](https://tensorflow.blog/kerasdl2/)의 예제 코드를 담고 있습니다.

<table align="left">
    <tr>
        <td>
            <a href="https://colab.research.google.com/github/rickiepark/deep-learning-with-python-2nd/blob/main/chapter11_part03_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
        </td>
    </tr>
</table>

## 트랜스포머 아키텍처

### 셀프 어텐션 이해하기

#### 일반화된 셀프 어텐션: 쿼리-키-값 모델

### 멀티 헤드 어텐션

### 트랜스포머 인코더

**데이터 가져오기**

In [3]:
# !rm -r aclImdb
# !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz
# !rm -r aclImdb/train/unsup

**데이터 준비**

In [4]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    if not os.path.exists(val_dir / category):
        os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 5244 files belonging to 2 classes.


2024-06-10 01:21:04.879114: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 01:21:04.907109: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 01:21:04.907142: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 01:21:04.909704: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 01:21:04.909735: I external/local_xla/xla/stream_executor

Found 19756 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**데이터 벡터화**

In [5]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

2024-06-10 01:21:45.566101: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


**`Layer` 층을 상속하여 구현한 트랜스포머 인코더**

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**트랜스포머 인코더를 사용하여 텍스트 분류하기**

In [7]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 256)      │     5,120,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,664,033 (21.61 MB)

 Trainable params: 5,664,033 (21.61 MB)

 Non-trainable params: 0 (0.00 B)

**트랜스포머 인코더 기반 모델 훈련하고 평가하기**

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20


I0000 00:00:1717982507.203491   62210 service.cc:145] XLA service 0x7f62f403fe80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1717982507.203539   62210 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-06-10 01:21:47.450620: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-10 01:21:47.876578: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1717982509.299676   62827 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_84', 188 bytes spill stores, 156 bytes spill loads

I0000 00:00:1717982509.428329   62832 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_80', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:17

  2/164 ━━━━━━━━━━━━━━━━━━━━ 13s 85ms/step - accuracy: 0.5391 - loss: 2.1686

I0000 00:00:1717982513.823930   62210 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


163/164 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.4878 - loss: 1.2097

...

164/164 ━━━━━━━━━━━━━━━━━━━━ 39s 192ms/step - accuracy: 0.4878 - loss: 1.2059 - val_accuracy: 0.5007 - val_loss: 0.7266
Epoch 2/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 22s 134ms/step - accuracy: 0.5212 - loss: 0.7142 - val_accuracy: 0.7125 - val_loss: 0.6202
Epoch 3/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 22s 132ms/step - accuracy: 0.6606 - loss: 0.6060 - val_accuracy: 0.7854 - val_loss: 0.4522
Epoch 4/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 22s 131ms/step - accuracy: 0.7872 - loss: 0.4474 - val_accuracy: 0.8215 - val_loss: 0.3952
Epoch 5/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 22s 132ms/step - accuracy: 0.8226 - loss: 0.3944 - val_accuracy: 0.8229 - val_loss: 0.3915
Epoch 6/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 22s 132ms/step - accuracy: 0.8658 - loss: 0.3116 - val_accuracy: 0.8274 - val_loss: 0.3862
Epoch 7/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 22s 133ms/step - accuracy: 0.8851 - loss: 0.2772 - val_accuracy: 0.8380 - val_loss: 0.3850
Epoch 8/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 21s 130ms/step - accuracy: 0.9005 - loss: 0.2321 - val

/data/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


777/782 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8374 - loss: 0.3811

I0000 00:00:1717982967.467528   68721 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_11', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1717982967.803138   68743 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_10', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1717982967.853141   68747 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_5', 24 bytes spill stores, 24 bytes spill loads

I0000 00:00:1717982967.903734   68735 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_5', 24 bytes spill stores, 24 bytes spill loads

I0000 00:00:1717982967.945888   68741 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_11', 108 bytes spill stores, 108 bytes spill loads

I0000 00:00:1717982967.956230   68748 asm_co

782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8374 - loss: 0.3811
테스트 정확도: 0.837


#### 위치 인코딩을 사용해 위치 정보 주입하기

**서브클래싱으로 위치 임베딩 구현하기**

In [12]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    # def compute_mask(self, inputs, mask=None):
    #     return tf.math.not_equal(inputs, 0)
def compute_mask(self, inputs, mask=None):
    return tf.keras.backend.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

#### 텍스트 분류 트랜스포머

**트랜스포머 인코더와 위치 임베딩 합치기**

In [13]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

# inputs = keras.Input(shape=(None,), dtype="int64")
inputs = keras.Input(shape=(sequence_length,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 600)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding_2          │ (None, 600, 256)       │     5,273,600 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_1           │ (None, 600, 256)       │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,817,633 (22.19 MB)

 Trainable params: 5,817,633 (22.19 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 22s 107ms/step - accuracy: 0.5419 - loss: 1.1295 - val_accuracy: 0.6746 - val_loss: 0.5955
Epoch 2/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 13s 81ms/step - accuracy: 0.7045 - loss: 0.5707 - val_accuracy: 0.7401 - val_loss: 0.5041
Epoch 3/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step - accuracy: 0.7978 - loss: 0.4274 - val_accuracy: 0.7760 - val_loss: 0.4570
Epoch 4/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 14s 82ms/step - accuracy: 0.8466 - loss: 0.3476 - val_accuracy: 0.8170 - val_loss: 0.4044
Epoch 5/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 13s 80ms/step - accuracy: 0.8744 - loss: 0.2980 - val_accuracy: 0.8148 - val_loss: 0.4095
Epoch 6/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 13s 81ms/step - accuracy: 0.9086 - loss: 0.2357 - val_accuracy: 0.8207 - val_loss: 0.3963
Epoch 7/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 13s 81ms/step - accuracy: 0.9253 - loss: 0.1941 - val_accuracy: 0.8095 - val_loss: 0.4526
Epoch 8/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 13s 80ms/step - accuracy: 0.9417 - loss: 0.1535 -

/data/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'positional_embedding_2', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/data/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:361: UserWarning: `build()` was called on layer 'transformer_encoder_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.8285 - loss: 0.3837
테스트 정확도: 0.825


### BoW 모델 대신 언제 시퀀스 모델을 사용하나요?